In [1]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration
from datasets import load_from_disk
import evaluate

/projectnb/cs640grp/students/chvskch/.conda/envs/proj_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# CHECKPOINT = "../Model_Checkpoints/closed-generative-qa/checkpoint-156" # Don't forget to replace this with the best checkpoint
CHECKPOINT = "google/flan-t5-base"

In [3]:
visa_qa_testing = load_from_disk("../Datasets/Visa_QA_V4/")["test"]

In [6]:
tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT)
model = T5ForConditionalGeneration.from_pretrained(CHECKPOINT)
def generate_output(data_point):
    question = data_point['question']
    prompt = f"You are an expert in dealing with questions of immigration and international travel. Answer the following question with proper reasoning and use the keywords to get some hints about the answer: Keywords: {', '.join(data_point['meta_tags'].split(','))} Question: {question}"
    encoded_input = tokenizer(prompt, return_tensors='pt')
    output = model.generate(**encoded_input)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    print("".join(output))
    return {"Predicted Answer": output}

In [7]:
prediction = visa_qa_testing.map(generate_output)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]/projectnb/cs640grp/students/chvskch/.conda/envs/proj_env/lib/python3.10/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Map:  20%|██        | 1/5 [00:00<00:01,  2.02 examples/s]

a.


Map:  40%|████      | 2/5 [00:00<00:01,  2.52 examples/s]

a.


Map:  60%|██████    | 3/5 [00:01<00:00,  2.09 examples/s]

airports have international transfer zones


Map:  80%|████████  | 4/5 [00:02<00:00,  1.38 examples/s]

The US doesn't check passports going out of the country, so they don't


Map: 100%|██████████| 5/5 [00:03<00:00,  1.63 examples/s]

['Expedia']


In [6]:
prediction

Dataset({
    features: ['meta_tags', 'question', 'answer', 'vote_score', 'Accepted', 'Predicted Answer'],
    num_rows: 5
})

In [7]:
rogue = evaluate.load("rouge")
predictions = prediction['Predicted Answer']
ground_truth = prediction['answer']
print(rogue.compute(predictions=predictions, references = ground_truth))

['I, the reader, are a permanent resident of Canada.', 'passport, hong-kong, indonesian-citizens', 'canada, license, driving-license', 'customs-and-immigration, covid-19, german-language, blue-', 'id, visas, customs-and-immigration, Malaysia']
["If I were you, I would just submit a new application for the family permit, including the passport copy.  The application is free of charge, after all.\nYour refusal even implies that you should do this (emphasis added):\n\nwhilst your application falls for refusal on the grounds of no sponsor ID I am satisfied that you are in a durable relationship akin to marriage in accordance with regulation 8 of the Immigration (EEA) Regulations 2016.\n\nIt's basically saying that the only reason they refused the application was the absence of the passport copy.\nYou can also fly to the UK and present your evidence at the border, as you suggest.  I suppose the chance of success is high, but the potential cost of failure is also high: you could be sent back